In [2]:
# Import libraries
import numpy as np
import pandas as pd
import category_encoders as cat_enc
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import pickle

In [3]:
# Import data
#path = "/Users/kreshnaa.raam/Documents/DR_Projects/FSDS/"
train = pd.read_csv("airline_delay_train.csv")

In [4]:

# Function for date based features
def date_features(data):
    data['FlightDate'] = pd.to_datetime(data['FlightDate'])
    data['day'] = pd.DatetimeIndex(data['FlightDate']).day.astype('category')
    data['month'] = pd.DatetimeIndex(data['FlightDate']).month.astype('category')
    data['year'] = pd.DatetimeIndex(data['FlightDate']).year.astype('category')
    data = data.drop('FlightDate', axis=1)
    data = data.drop('DepTime', axis=1)
    return data

# Apply FE on train data
train = date_features(train)

In [5]:

# Assign numeric and categorical variable list
numerical_cols = ['Distance']
categorical_cols = ['UniqueCarrier', 'Origin', 'Dest', 'Day_of_Week', 'year', 'month', 'day']

# Store predictors and target in two different variables
def split_train_data(data):
    y = data['dep_delayed_15min']
    X = data.drop('dep_delayed_15min', axis=1)
    return X, y

X, y = split_train_data(train)

In [6]:


# Apply train test split with 25% data for validation
X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=0.25)

# Creating ss transformer to scale the continuous numerical data with StandardScaler()
numerical_transformer = Pipeline(steps=[('ss', StandardScaler())])

# Creating categorical transformer using ordinal encoder
categorical_transformer = Pipeline(steps=[('ordinal', cat_enc.OrdinalEncoder())])

# Creating preprocess column transformer to combine the ss and ohe pipelines
preprocess = ColumnTransformer(
    transformers=[
        ('cont', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

In [7]:
# Creat classifier pipeline using random forest
classifier_pipe = Pipeline(steps=[('preprocessor', preprocess),
                                  ('classifier', RandomForestClassifier())])

# Set hyperparameter index
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start=800, stop=1000, num=2)]
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 20, num=3)]
# Minimum number of samples required to split a node
min_samples_split = [10, 20]

# Create hyperparameter grid for CV Search for best model
hyper_param_grid = {'classifier__n_estimators': n_estimators,
                    'classifier__max_depth': max_depth,
                    'classifier__min_samples_split': min_samples_split}

In [8]:

# Set grid search using roc_auc optimization with 3 fold cv
CV = GridSearchCV(classifier_pipe, hyper_param_grid, n_jobs=-1, scoring='roc_auc', verbose=2, cv=3)

# fit the model
CV.fit(X_train, y_train)


Fitting 3 folds for each of 12 candidates, totalling 36 fits


/opt/conda/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('cont',
                                                                         Pipeline(steps=[('ss',
                                                                                          StandardScaler())]),
                                                                         ['Distance']),
                                                                        ('cat',
                                                                         Pipeline(steps=[('ordinal',
                                                                                          OrdinalEncoder())]),
                                                                         ['UniqueCarrier',
                                                                          'Origin',
                                                                          'Dest',


In [9]:

# Predict on validation data and generate scores
target_names = y_validation.unique().astype(str)
y_pred = CV.predict(X_validation)
print(classification_report(y_validation, y_pred, target_names=target_names))
print("{}{}".format("Cross - Validation: ", CV.best_score_))
print("{}{}".format("Validation: ", CV.score(X_validation, y_validation)))


              precision    recall  f1-score   support

           1       0.83      0.98      0.90     82047
           0       0.65      0.13      0.22     19465

    accuracy                           0.82    101512
   macro avg       0.74      0.56      0.56    101512
weighted avg       0.79      0.82      0.77    101512

Cross - Validation: 0.7298930524208348
Validation: 0.7344859318306373


In [10]:

print('Reading test df')
test = pd.read_csv(path + "airline_delay_test - airline_delay_test_new.csv")
test = date_features(test)
X, y = split_train_data(test)
print("{}{}".format("Holdout: ", CV.score(X,y)))

Reading test df


NameError: name 'path' is not defined

In [11]:

#pickle.dump(CV, open('model_randomForest_v1.sav', 'wb'))


with open('customf.pkl', 'wb') as pkl:
    pickle.dump(CV, pkl)

